In [3]:
import os
import cv2
import psutil
from tkinter import Tk
from tkinter.filedialog import askdirectory
from tqdm.notebook import tqdm

p = psutil.Process(os.getpid())
p.cpu_affinity([0, 1, 2, 3, 4, 5, 6, 7])

start_time = 11 #sec
duration = 1 #min
video_fps = 120

# 開啟資料夾對話框，讓使用者選擇資料夾
Tk().withdraw()
foldername = askdirectory(title='選擇資料夾')

# 取得資料夾內的所有影片檔案
video_files = [f for f in os.listdir(foldername) if f.endswith('.MP4')]
# 輸出資料夾設定
output_dir = askdirectory(title='選擇輸出資料夾')

for filename in tqdm(video_files, desc='total_file'):
   # 影片檔案的完整路徑
    filepath = os.path.join(foldername, filename)

    # 讀取影片
    cap = cv2.VideoCapture(filepath)

    # 獲取原始影片的尺寸
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # 新的裁切後的尺寸
    new_width = 1150 
    new_height = height // 4  # 裁去上四分之一和下四分之一

    # 計算裁切後的上下邊界
    top = 30
    bottom = top + new_height-80
    left = 380
    right = new_width+left

    new_width = int((right - left)//1.33)
    new_height = int((bottom - top)//1.33)

    # 輸出影片設定
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    filename_without_ext = os.path.splitext(filename)[0]
    output_filename = os.path.join(output_dir, f"{filename_without_ext}_cropped.mp4")
    output = cv2.VideoWriter(output_filename, fourcc, fps, (new_width, new_height), isColor=False)

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    end_frame = (60*duration)*video_fps
    start_frames = start_time*video_fps
    now_frames = start_frames

    cap.set(cv2.CAP_PROP_POS_FRAMES, int(start_frames))
    gpu_frame = cv2.cuda_GpuMat()

    # 使用 tqdm 顯示進度條
    for now_frames in tqdm(range(end_frame), desc=filename):  # 使用tqdm迭代
        
        ret, frame = cap.read()
    
        # 當讀取完所有幀，退出循環
        if not ret or now_frames >= end_frame :
            break
        
        if now_frames >= end_frame :    
            print('break')

        # 裁切畫面大小
        cropped_frame = frame[top:bottom, left:right]

        # 進行下採樣
        resized_image = cv2.resize(cropped_frame, (new_width, new_height))

        # 進行灰階轉換（可選）
        gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

        # 使用高斯模糊進行影像平滑化
        blurred_image = cv2.GaussianBlur(gray_image, (0, 0), 3)
        
        # 計算Unsharp Mask濾波器
        output_image = cv2.addWeighted(gray_image, 2, blurred_image, -1, 0)

        # # 輸出裁切後的影片
        output.write(output_image)

        # # # 顯示裁切後的畫面
        # cv2.imshow('unsharp_mask', output_image)

        # # 按下'q'鍵退出
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break

    # 釋放資源
    cap.release()
    output.release() 

cv2.destroyAllWindows()

total_file:   0%|          | 0/19 [00:00<?, ?it/s]

6L_5.MP4:   0%|          | 0/7200 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [1]:
import os
import cv2
import psutil
from tkinter import Tk
from tkinter.filedialog import askdirectory
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed


# Parameters
start_time = 11  # sec
duration = 30  # min
video_fps = 120

# Open folder dialog to select folder
Tk().withdraw()
foldername = askdirectory(title='選擇資料夾')

# Get all video files in the folder
video_files = [f for f in os.listdir(foldername) if f.endswith('.MP4')]
# Set output folder
output_dir = askdirectory(title='選擇輸出資料夾')

# Function to process each video file
def process_video(filename):
    try:
        filepath = os.path.join(foldername, filename)
        cap = cv2.VideoCapture(filepath)

        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        new_width = 1150
        new_height = height // 4

        top = 30
        bottom = top + new_height - 80
        left = 380
        right = new_width + left

        new_width = int((right - left) // 1.33)
        new_height = int((bottom - top) // 1.33)

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        fps = cap.get(cv2.CAP_PROP_FPS)
        filename_without_ext = os.path.splitext(filename)[0]
        output_filename = os.path.join(output_dir, f"{filename_without_ext}_cropped.mp4")
        output = cv2.VideoWriter(output_filename, fourcc, fps, (new_width, new_height), isColor=False)

        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        end_frame = (60 * duration) * video_fps
        start_frames = start_time * video_fps
        now_frames = start_frames

        cap.set(cv2.CAP_PROP_POS_FRAMES, int(start_frames))

        for now_frames in tqdm(range(end_frame), desc=filename):
            ret, frame = cap.read()

            if not ret or now_frames >= end_frame:
                break

            cropped_frame = frame[top:bottom, left:right]
            resized_image = cv2.resize(cropped_frame, (new_width, new_height))
            gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
            blurred_image = cv2.GaussianBlur(gray_image, (0, 0), 3)
            output_image = cv2.addWeighted(gray_image, 2, blurred_image, -1, 0)

            output.write(output_image)

        cap.release()
        output.release()
    except Exception as e:
        print(f"Error processing {filename}: {e}")
    finally:
        cv2.destroyAllWindows()

if __name__ == '__main__':
    with ThreadPoolExecutor(max_workers=psutil.cpu_count(logical=False)) as executor:
        futures = [executor.submit(process_video, filename) for filename in video_files]
        for future in as_completed(futures):
            future.result()

6L_5.MP4:   0%|          | 0/216000 [00:00<?, ?it/s]

6F_6.MP4:   0%|          | 0/216000 [00:00<?, ?it/s]

6K_10.MP4:   0%|          | 0/216000 [00:00<?, ?it/s]

6B_10.MP4:   0%|          | 0/216000 [00:00<?, ?it/s]

3H_7.MP4:   0%|          | 0/216000 [00:00<?, ?it/s]

...

4A_7.MP4:   0%|          | 0/216000 [00:00<?, ?it/s]

4C_06.MP4:   0%|          | 0/216000 [00:00<?, ?it/s]

4G_8.MP4:   0%|          | 0/216000 [00:00<?, ?it/s]

2B_8.MP4:   0%|          | 0/216000 [00:00<?, ?it/s]

6E_7.MP4:   0%|          | 0/216000 [00:00<?, ?it/s]